## Park_HeeJI_HW07.ipynb
- Student Name: Hee Ji Park
- Github username : heejipark 
- USC ID: 4090715830
- Contact Email: heejipar@usc.edu

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### Required libraries

In [16]:
import pandas as pd
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from collections import defaultdict
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

### (a) Download the Anuran Calls (MFCCs) Data Set. Choose 70% of the data randomly as the training set.

In [2]:
df = pd.read_csv('./../data/Frogs_MFCCs.csv')
train_data, test_data = train_test_split(df, train_size=0.7, shuffle=True)
print(" The number of whole data : %d\n \
The number of train data : %d\n \
The number of test data : %d" %(len(df), len(train_data), len(test_data)))

 The number of whole data : 7195
 The number of train data : 5036
 The number of test data : 2159


In [3]:
train_data.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
2371,1.0,0.176844,0.064955,0.443387,0.123255,-0.101175,-0.148219,0.153857,0.283427,-0.003231,...,0.093488,-0.097267,-0.101748,0.020772,0.188832,0.117297,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18
532,1.0,0.523367,0.631220,0.527269,-0.116156,-0.055174,0.431538,0.064446,-0.344358,0.155121,...,0.027519,0.162381,-0.046196,-0.044553,0.122022,0.041092,Leptodactylidae,Adenomera,AdenomeraAndre,8
6534,1.0,0.219132,0.328145,0.276312,0.138941,0.161799,0.069704,-0.098882,-0.023238,0.060825,...,0.089075,0.012570,0.005598,0.012546,-0.020321,0.014343,Hylidae,Hypsiboas,HypsiboasCordobae,43
5146,1.0,0.607690,0.735682,0.388962,-0.137170,0.160811,0.137000,0.133233,0.138852,-0.127371,...,-0.109483,-0.006866,-0.078918,0.001121,0.049807,-0.205169,Hylidae,Hypsiboas,HypsiboasCinerascens,36
5719,1.0,0.324770,0.517765,0.352570,0.124584,0.139990,0.048961,-0.053079,0.167065,0.130890,...,-0.014846,-0.051318,0.066071,-0.013817,-0.043170,0.118018,Hylidae,Hypsiboas,HypsiboasCordobae,40


In [4]:
test_data.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
3401,1.0,0.011335,-0.124341,0.296202,0.122151,0.074736,0.006696,0.052122,0.262498,0.167792,...,0.199462,0.086548,-0.122710,-0.186576,0.002982,0.235526,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,21
2796,1.0,0.488668,0.300977,0.506851,0.177675,0.113348,-0.054620,-0.002175,0.278806,-0.109099,...,0.110031,-0.048000,-0.080400,-0.054633,0.074748,0.185410,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,19
489,1.0,0.516489,0.560199,0.478810,-0.124926,-0.026845,0.406056,0.063808,-0.251093,0.124204,...,0.073405,0.134755,-0.055265,-0.050070,0.103260,0.029075,Leptodactylidae,Adenomera,AdenomeraAndre,8
3906,1.0,0.689811,0.408619,0.539722,0.168304,0.045319,-0.051021,0.129722,0.250857,0.089355,...,0.144877,-0.009924,-0.100310,-0.121318,0.028429,0.157641,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,22
6345,1.0,0.554001,0.436325,0.227448,0.046845,0.138826,0.052548,-0.040940,0.117689,0.070114,...,-0.026022,-0.013210,0.028749,-0.042943,-0.019980,0.061764,Hylidae,Hypsiboas,HypsiboasCordobae,42


### (b) 
Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance).

#### i. Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.

<b>Answer : </b>

1. Exact Matching Ration (EMR): The Exact Matching Ratio evaluation metric extends the concept of accuracy from the single-label classification problem to a multi-label classification problem. One trivial way around would just to ignore partially correct (consider them incorrect) and extend the accuracy used in single label case for multi-label prediction. Clearly, a disadvantage of this measure is that it does not distinguish between complete incorrect and partially correct which might be considered harsh.
<img src='./../data/img/EMR.PNG' width=300>

2. Hamming Loss : The fraction of the wrong labels to the total number of labels. It means how many times on average, the relevance of an example to a class label is incorrectly predicted. Therefore, hamming loss takes into account the prediction error (an incorrect label is predicted) and missing error (a relevant label not predicted), normalized over total number of classes and total number of examples. Ideally, we would expect the hamming loss to be 0, which would imply no error; practically the smaller the value of hamming loss, the better the performance of the learning algorithm.
<img src='./../data/img/Hamming.PNG' width=400>

3. Hamming Score : The fraction of labels that are correctly predicted = 1 - hamming_loss

In [5]:
def make_To_Dic(category):
    categories_To_int = {}
    for i, c in enumerate(test_Y[category].unique()):
        categories_To_int[c] = i
    return categories_To_int

In [6]:
def hamming_score(test_Y, category, y_pred):
    return 1 - hamming_loss(test_Y[category], y_pred)

In [7]:
categories = ['Family', 'Genus', 'Species']

In [8]:
train_X, train_Y = train_data.iloc[:, :-4], train_data.loc[:,categories]
test_X, test_Y = test_data.iloc[:, :-4], test_data.loc[:,categories]

#### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation.

#### With raw attributes + the weight of the SVM penalty + the width of the Gaussian Kernel 

In [9]:
TOT_SVM_EMR = []
TOT_SVM_HammingL = []
TOT_SVM_HammingS = []
print("With raw attributes \n")

for category in categories:
    print("--------------------------------------------------------------------\n")
    print("Category : %s \n" %category)
    parameters = {'estimator__C':np.logspace(-3, 6, 10), 'estimator__gamma':np.linspace(0.1, 2, 10), 'estimator__kernel': ['rbf']}
    clf = OneVsRestClassifier(SVC())
    clf = GridSearchCV(clf, param_grid=parameters, cv=StratifiedKFold(10)).fit(train_X, train_Y[category])

    # Completed get the best estimator.
    # Refit the model and predict with test data set
    SVMmodel = clf.best_estimator_
    SVMmodel = SVMmodel.fit(train_X, train_Y[category])
    y_pred = SVMmodel.predict(test_X)
    
    EMR = accuracy_score(test_Y[category], y_pred)
    HammingL = hamming_loss(test_Y[category], y_pred)
    HammingS = hamming_score(test_Y, category, y_pred)
    TOT_SVM_EMR.append(EMR)
    TOT_SVM_HammingL.append(HammingL)
    TOT_SVM_HammingS.append(HammingS)
    
    print("\n")
    print('\t - Best Parameter {}'.format(clf.best_params_))
    print('\t - Exact Match score is %f'% EMR)
    print('\t - Hamming Loss is %f'% HammingL)
    print('\t - Hamming Score is %f'% HammingS)
    
    print("\n\n")

With raw attributes 

--------------------------------------------------------------------

Category : Family 



	 - Best Parameter {'estimator__C': 10.0, 'estimator__gamma': 2.0, 'estimator__kernel': 'rbf'}
	 - Exact Match score is 0.992589
	 - Hamming Loss is 0.007411
	 - Hamming Score is 0.992589



--------------------------------------------------------------------

Category : Genus 



	 - Best Parameter {'estimator__C': 10.0, 'estimator__gamma': 2.0, 'estimator__kernel': 'rbf'}
	 - Exact Match score is 0.991200
	 - Hamming Loss is 0.008800
	 - Hamming Score is 0.991200



--------------------------------------------------------------------

Category : Species 



	 - Best Parameter {'estimator__C': 10.0, 'estimator__gamma': 2.0, 'estimator__kernel': 'rbf'}
	 - Exact Match score is 0.990273
	 - Hamming Loss is 0.009727
	 - Hamming Score is 0.990273





In [10]:
# Average EMR / Hamming Loss / Hamming Score
print("Average Exact Match Score with raw attributes : %f" % np.mean(TOT_SVM_EMR))
print("Average Hamming Loss with raw attributes : %f" % np.mean(TOT_SVM_HammingL))
print("Average Hamming Score with raw attributes : %f" % np.mean(TOT_SVM_HammingS))

Average Exact Match Score with raw attributes : 0.991354
Average Hamming Loss with raw attributes : 0.008646
Average Hamming Score with raw attributes : 0.991354


<B> Result report -> </B>  The results are summarized in the above print statement.

#### iii. Repeat 1(b)ii with L1-penalized SVMs. Remember to standardize the attributes. Determine the weight of the SVM penalty using 10 fold cross validation.

In [11]:
# Standardized attributes
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale = scaler.transform(test_X)

In [12]:
TOT_L1_SVM_EMR = []
TOT_L1_SVM_HammingL = []
TOT_L1_SVM_HammingS = []

print("L1-penalized SVM with standardized attributes \n")

for category in categories:
    print("--------------------------------------------------------------------\n")
    print("Category : %s \n" % category)
    parameters = {'C':np.logspace(-3, 6, 10)}
    clf = LinearSVC(penalty='l1', dual=False, max_iter=20000)
    clf = GridSearchCV(clf, param_grid=parameters, cv=StratifiedKFold(10)).fit(train_X_scale, train_Y[category])
    
    # Completed get the best estimator.
    # Refit the model and predict with test data set
    l1SVMmodel = clf.best_estimator_
    l1SVMmodel = l1SVMmodel.fit(train_X_scale, train_Y[category])
    y_pred = l1SVMmodel.predict(test_X_scale)
    
    EMR = accuracy_score(test_Y[category], y_pred)
    HammingL = hamming_loss(test_Y[category], y_pred)
    HammingS = hamming_score(test_Y, category, y_pred)
    
    TOT_L1_SVM_EMR.append(EMR)
    TOT_L1_SVM_HammingL.append(HammingL)
    TOT_L1_SVM_HammingS.append(HammingS)
    
    print("\n")
    print('\t - Best Parameter {}'.format(clf.best_params_))
    print('\t - Exact Match score is %f'% EMR)
    print('\t - Hamming Loss is %f'% HammingL)
    print('\t - Hamming Score is %f'% HammingS)
    print("\n\n")

L1-penalized SVM with standardized attributes 

--------------------------------------------------------------------

Category : Family 



	 - Best Parameter {'C': 1.0}
	 - Exact Match score is 0.935618
	 - Hamming Loss is 0.064382
	 - Hamming Score is 0.935618



--------------------------------------------------------------------

Category : Genus 



	 - Best Parameter {'C': 1.0}
	 - Exact Match score is 0.949977
	 - Hamming Loss is 0.050023
	 - Hamming Score is 0.949977



--------------------------------------------------------------------

Category : Species 



	 - Best Parameter {'C': 10.0}
	 - Exact Match score is 0.960630
	 - Hamming Loss is 0.039370
	 - Hamming Score is 0.960630





In [13]:
# Average EMR / Hamming Loss / Hamming Score
print("Average Exact Match Score with Standardized attributes : %f" % np.mean(TOT_L1_SVM_EMR))
print("Average Hamming Loss with Standardized attributes : %f" % np.mean(TOT_L1_SVM_HammingL))
print("Average Hamming Score with Standardized attributes : %f" % np.mean(TOT_L1_SVM_HammingS))

Average Exact Match Score with Standardized attributes : 0.948742
Average Hamming Loss with Standardized attributes : 0.051258
Average Hamming Score with Standardized attributes : 0.948742


<B> Result report -> </B>  The results are summarized in the above print statement.

#### iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. 

In [14]:
# Standardized attributes
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale = scaler.transform(test_X)

In [17]:
TOT_L1_SMOTE_SVM_EMR = []
TOT_L1_SMOTE_SVM_HammingL = []
TOT_L1_SMOTE_SVM_HammingS = []

print("L1-penalized SVM with standardized attributes and SMOTE \n")

for category in categories:
    print("--------------------------------------------------------------------\n")
    print("Category : %s \n" % category)
    parameters = {'classification__C':np.logspace(-3, 6, 5)}
    smote = SMOTE()
    clf = LinearSVC(penalty='l1', dual=False, max_iter=20000)
    pipe = Pipeline(steps=[('sampling', smote), ('classification', clf)])
    clf = GridSearchCV(pipe, param_grid=parameters, cv=KFold(10)).fit(train_X_scale, train_Y[category])
    
    # Completed get the best estimator.
    # Refit the model and predict with test data set
    l1SVMmodel = clf.best_estimator_
    l1SVMmodel = l1SVMmodel.fit(train_X_scale, train_Y[category])
    y_pred = l1SVMmodel.predict(test_X_scale)
    
    EMR = accuracy_score(test_Y[category], y_pred)
    HammingL = hamming_loss(test_Y[category], y_pred)
    HammingS = hamming_score(test_Y, category, y_pred)
    
    TOT_L1_SMOTE_SVM_EMR.append(EMR)
    TOT_L1_SMOTE_SVM_HammingL.append(HammingL)
    TOT_L1_SMOTE_SVM_HammingS.append(HammingS)
    
    print("\n")
    print('\t - Best Parameter {}'.format(clf.best_params_))
    print('\t - Exact Match score is %f'% EMR)
    print('\t - Hamming Loss is %f'% HammingL)
    print('\t - Hamming Score is %f'% HammingS)
    print("\n\n")

L1-penalized SVM with standardized attributes and SMOTE 

--------------------------------------------------------------------

Category : Family 



	 - Best Parameter {'classification__C': 5623.413251903491}
	 - Exact Match score is 0.917091
	 - Hamming Loss is 0.082909
	 - Hamming Score is 0.917091



--------------------------------------------------------------------

Category : Genus 



	 - Best Parameter {'classification__C': 0.1778279410038923}
	 - Exact Match score is 0.905512
	 - Hamming Loss is 0.094488
	 - Hamming Score is 0.905512



--------------------------------------------------------------------

Category : Species 



	 - Best Parameter {'classification__C': 1000000.0}
	 - Exact Match score is 0.958314
	 - Hamming Loss is 0.041686
	 - Hamming Score is 0.958314





In [18]:
# Average EMR / Hamming Loss / Hamming Score
print("Average Exact Match Score with Standardized attributes : %f" % np.mean(TOT_L1_SMOTE_SVM_EMR))
print("Average Hamming Loss with Standardized attributes : %f" % np.mean(TOT_L1_SMOTE_SVM_HammingL))
print("Average Hamming Score with Standardized attributes : %f" % np.mean(TOT_L1_SMOTE_SVM_HammingS))

Average Exact Match Score with Standardized attributes : 0.926972
Average Hamming Loss with Standardized attributes : 0.073028
Average Hamming Score with Standardized attributes : 0.926972


<B> Result report -> </B>  The results are summarized in the above print statement.

#### Report your conclusions about the classifiers you trained.

In [19]:
report = pd.DataFrame(columns = ['SVM with raw attributes','L1-penalized SVMs','L1-penalized SVMs using SMOTE'],
                      index=['Average Exact Match Score', 'Average Hamming Loss', 'Average Hamming Score'])
report.loc['Average Exact Match Score', 'SVM with raw attributes'] = np.mean(TOT_SVM_EMR)
report.loc['Average Exact Match Score', 'L1-penalized SVMs'] = np.mean(TOT_L1_SVM_EMR)
report.loc['Average Exact Match Score', 'L1-penalized SVMs using SMOTE'] = np.mean(TOT_L1_SMOTE_SVM_EMR)

report.loc['Average Hamming Loss', 'SVM with raw attributes'] = np.mean(TOT_SVM_HammingL)
report.loc['Average Hamming Loss', 'L1-penalized SVMs'] = np.mean(TOT_L1_SVM_HammingL)
report.loc['Average Hamming Loss', 'L1-penalized SVMs using SMOTE'] = np.mean(TOT_L1_SMOTE_SVM_HammingL)

report.loc['Average Hamming Score', 'SVM with raw attributes'] = np.mean(TOT_SVM_HammingS)
report.loc['Average Hamming Score', 'L1-penalized SVMs'] = np.mean(TOT_L1_SVM_HammingS)
report.loc['Average Hamming Score', 'L1-penalized SVMs using SMOTE'] = np.mean(TOT_L1_SMOTE_SVM_HammingS)
report

,SVM with raw attributes,L1-penalized SVMs,L1-penalized SVMs using SMOTE
Average Exact Match Score,0.991354,0.948742,0.926972
Average Hamming Loss,0.008646,0.051258,0.073028
Average Hamming Score,0.991354,0.948742,0.926972


<b> Answer: </b>
- As shown in the table above, it can be seen that the SVM classifier model using Gausian Kernel provides the best results with the lowest Hamming Loss and the highest Exact Matching Score.

### 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
Monte-Carlo Simulation: Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate.


### (a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set. 
(do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.

- Note that silhouette_score requires more than 1 cluster labels. This causes the error. So, I use the range (2, 51) instead of (1, 51)

In [21]:
data = df.iloc[:, :-4]
BestK = {}
BestK_labels = {}
for num in range(1, 51):
    print("%d iteration --"% num)
    silhouetteScore = {}
    Klabels = {}
    for k in range(2, 51):
        kmeans = KMeans(n_clusters=k).fit(data)
        Klabels[k] = kmeans.labels_
        silhouetteScore[k] = silhouette_score(data, kmeans.labels_)
    
    silhouetteScore = sorted(silhouetteScore.items(), key=lambda item: item[1], reverse=True)
    BestK['Iteration ' + str(num)] = silhouetteScore[0][0]
    BestK_labels['Iteration ' + str(num)] = Klabels[silhouetteScore[0][0]]

1 iteration --
2 iteration --
3 iteration --
4 iteration --
5 iteration --
6 iteration --
7 iteration --
8 iteration --
9 iteration --
10 iteration --
11 iteration --
12 iteration --
13 iteration --
14 iteration --
15 iteration --
16 iteration --
17 iteration --
18 iteration --
19 iteration --
20 iteration --
21 iteration --
22 iteration --
23 iteration --
24 iteration --
25 iteration --
26 iteration --
27 iteration --
28 iteration --
29 iteration --
30 iteration --
31 iteration --
32 iteration --
33 iteration --
34 iteration --
35 iteration --
36 iteration --
37 iteration --
38 iteration --
39 iteration --
40 iteration --
41 iteration --
42 iteration --
43 iteration --
44 iteration --
45 iteration --
46 iteration --
47 iteration --
48 iteration --
49 iteration --
50 iteration --


In [22]:
# Print Best K for each iteration
pd.DataFrame.from_dict(BestK, orient='index', columns=['Best K'])

,Best K
Iteration 1,4
Iteration 2,4
Iteration 3,4
Iteration 4,4
Iteration 5,4
Iteration 6,4
Iteration 7,4
Iteration 8,4
Iteration 9,4
Iteration 10,4


### (b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.

In [24]:
Y = df.loc[:,categories] # labels
totCluster = []

for num in range(1, 51):
    find = 'Iteration ' + str(num)
    k = BestK[find] # the number of K
    label = BestK_labels[find]
    for cluster in range(0, k):
        choose = {}
        Family = defaultdict(int)
        Genus = defaultdict(int)
        Species = defaultdict(int)
        for idx in range(len(Y)):
            if cluster == label[idx]:
                addFamilyClass = Y['Family'][idx]
                Family[addFamilyClass] += 1
                
                addGenusClass = Y['Genus'][idx]
                Genus[addGenusClass] += 1
                
                addSpeciesClass = Y['Species'][idx]
                Species[addSpeciesClass] += 1
                
        majority_Family = sorted(Family.items(), key=lambda item: item[1], reverse=True)
        majority_Genus = sorted(Genus.items(), key=lambda item: item[1], reverse=True)
        majority_Species = sorted(Species.items(), key=lambda item: item[1], reverse=True)
        
        choose['Iteration'] = find
        choose['Cluster'] = cluster
        choose['Family'] = majority_Family[0][0]
        choose['Genus'] = majority_Genus[0][0]
        choose['Species'] = majority_Species[0][0]
        
        totCluster.append(choose)

In [25]:
predDF = pd.DataFrame(totCluster)
predDF

,Iteration,Cluster,Family,Genus,Species
0,Iteration 1,0,Leptodactylidae,Adenomera,AdenomeraAndre
1,Iteration 1,1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Iteration 1,2,Hylidae,Hypsiboas,HypsiboasCordobae
3,Iteration 1,3,Hylidae,Hypsiboas,HypsiboasCordobae
4,Iteration 2,0,Dendrobatidae,Ameerega,Ameeregatrivittata
...,...,...,...,...,...
195,Iteration 49,3,Hylidae,Hypsiboas,HypsiboasCordobae
196,Iteration 50,0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
197,Iteration 50,1,Dendrobatidae,Ameerega,Ameeregatrivittata
198,Iteration 50,2,Hylidae,Hypsiboas,HypsiboasCordobae


In [26]:
BestK_labels

{'Iteration 1': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 2': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Iteration 3': array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'Iteration 4': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 5': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 6': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Iteration 7': array([3, 3, 3, ..., 3, 3, 3], dtype=int32),
 'Iteration 8': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 9': array([3, 3, 3, ..., 3, 3, 3], dtype=int32),
 'Iteration 10': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 11': array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'Iteration 12': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 13': array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 'Iteration 14': array([2, 2, 2, ..., 2, 2, 2], dtype=int32),
 'Iteration 15': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'Iteration 16': array([3, 3, 3, ..., 3, 3, 3], dtype=int32),
 'Iteration 17': 

### (c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance, Hamming score, and Hamming loss between the true labels and the labels assigned by clusters.

In [27]:
def hamming_distance(true_Family, true_Genus, true_Species, pred_Family, pred_Genus, pred_Species):
    trueLabel = []
    predLabel = []
    for f, g, s in zip(true_Family, true_Genus, true_Species):
        trueLabel.append((f + ' ' + g + ' ' + s))

    for f, g, s in zip(pred_Family, pred_Genus, pred_Species):
        predLabel.append((f + ' ' + g + ' ' + s))  

    mismatch = 0
    for t, p in zip(trueLabel, predLabel):
        if t != p:
            mismatch += 1

    return mismatch

In [30]:
tot_Hamming = {}
for num in range(1, 51):
    find = 'Iteration ' + str(num)
    pred_Family = []
    pred_Genus = []
    pred_Species = []
    iteration = {}
    
    for idx, cl in enumerate(BestK_labels[find]):
        pred_Row = predDF[(predDF['Iteration'] == find) & (predDF['Cluster'] == cl)]
        pred_Family.append(pred_Row['Family'].item())
        pred_Genus.append(pred_Row['Genus'].item())
        pred_Species.append(pred_Row['Species'].item())
        
        
    true_Family = np.array(Y['Family'])
    true_Genus = np.array(Y['Genus'])
    true_Species = np.array(Y['Species'])

    pred_Family = np.array(pred_Family)
    pred_Genus = np.array(pred_Genus)
    pred_Species = np.array(pred_Species)

    
    HammingL_Family = hamming_loss(true_Family, pred_Family)
    HammingS_Family = accuracy_score(true_Family, pred_Family)

    HammingL_Genus = hamming_loss(true_Genus, pred_Genus)
    HammingS_Genus = accuracy_score(true_Genus, pred_Genus)

    HammingL_Species = hamming_loss(true_Species, pred_Species)
    HammingS_Species = accuracy_score(true_Species, pred_Species)
    
    Hamming_distance = hamming_distance(true_Family, true_Genus, true_Species, pred_Family, pred_Genus, pred_Species)
    
    avg_HammingL = (HammingL_Family + HammingL_Genus + HammingL_Species)/3
    avg_HammingS = (HammingS_Family + HammingS_Genus + HammingS_Species)/3
    
    iteration['Hamming Distance'] = Hamming_distance
    iteration['Hamming Loss'] = avg_HammingL
    iteration['Hamming Score'] = avg_HammingS
    
    tot_Hamming[find] = iteration

In [31]:
# Print Best K for each iteration
hammingDF = pd.DataFrame.from_dict(tot_Hamming, orient='index')
hammingDF

,Hamming Distance,Hamming Loss,Hamming Score
Iteration 1,2281,0.245124,0.754876
Iteration 2,1759,0.222423,0.777577
Iteration 3,1759,0.222423,0.777577
Iteration 4,1753,0.221774,0.778226
Iteration 5,2242,0.280056,0.719944
Iteration 6,1759,0.222423,0.777577
Iteration 7,1961,0.233727,0.766273
Iteration 8,1760,0.222469,0.777531
Iteration 9,1759,0.222423,0.777577
Iteration 10,1759,0.222423,0.777577


In [32]:
report = pd.DataFrame(columns = hammingDF.columns, index=['average', 'standard deviation'])
report.loc['average', 'Hamming Distance'] = np.mean(hammingDF['Hamming Distance'])
report.loc['average', 'Hamming Loss'] = np.mean(hammingDF['Hamming Loss'])
report.loc['average', 'Hamming Score'] = np.mean(hammingDF['Hamming Score'])
report.loc['standard deviation', 'Hamming Distance'] = np.std(hammingDF['Hamming Distance'])
report.loc['standard deviation', 'Hamming Loss'] = np.std(hammingDF['Hamming Loss'])
report.loc['standard deviation', 'Hamming Score'] = np.std(hammingDF['Hamming Score'])
report

,Hamming Distance,Hamming Loss,Hamming Score
average,1803.36,0.225096,0.774904
standard deviation,141.097804,0.009646,0.009646


## 3. ISLR 12.6.2

<img src='./../data/ISLR/ISLR-12-6-2.png'>

### Reference
- Multi-label : https://www.linkedin.com/pulse/hamming-score-multi-label-classification-chandra-sharat/
- Hamming score : https://stackoverflow.com/questions/32239577/getting-the-accuracy-for-multi-label-prediction-in-scikit-learn
- GridSearch : https://www.datatechnotes.com/2019/09/how-to-use-gridsearchcv-in-python.html
- LinearSVC : https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
- OneVsRestClassifier : https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html
- Pipeline : https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html